# 1. Recall

### 1.1 Recall: Transformer

<img src='attachment/43. Full Architecture of Transformer.png' style='zoom:50%'/>

### 1.2 Recall: BERT

<img src='attachment/47. Recall_BERT.png' style='zoom:40%'/>

### 1.3 Recall: Language Model

- For the sequence $\color{MediumOrchid}{w_{1}, w_{2}, \ldots, w_{n}}$, using the chain rule, we have: 

$$
P\left(w_{1}, \ldots, w_{n}\right)=P\left(w_{n} \mid w_{1}, \ldots, w_{n-1}\right) P\left(w_{n-1} \mid w_{1}, \ldots, w_{n-2}\right) \ldots P\left(w_{2} \mid w_{1}\right) P\left(w_{1}\right)
$$

- N-Gram Approximation: $\color{MediumOrchid}{P\left(w_{1}, \ldots, w_{n}\right)=\prod_{i=1}^{n} P\left(w_{i} \mid w_{i-N+1}, \ldots, w_{i-1}\right)}$

- Applications:
    - Machine Translation: $\color{MediumOrchid}{P(\text{the cat is small })>P(\text{ small is the cat} )}$
    - Grammar Checking: $\color{MediumOrchid}{P(\text{ He graduated from SJTU. )>P(He graduated on SJTU.)}}$

# 2. GPT

GPT: Generative Pre-Training

相关论文:

1. Radford, A., & Narasimhan, K. (2018). Improving Language Understanding by Generative Pre-Training.

2. Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2019). Language Models are Unsupervised Multitask Learners.

3. Brown, T. et al. “Language Models are Few-Shot Learners.” ArXiv abs/2005.14165 (2020): n. pag.

### 2.1 Introduction

1. In this paper, we explore a semi-supervised approach for language understanding tasks using a combination of <font color=red>unsupervised pre-training and supervised fine-tuning.</font>


2. Our goal is to learn a <font color=red>universal representation</font> that transfers with little adaptation to a wide range of tasks.


3. We employ a two-stage training procedure. 
    - First, we use a language modeling objective on the unlabeled data to learn the initial parameters of a neural network model. 
    - Subsequently, we adapt these parameters to a target task using the corresponding supervised objective.
    
    
4. For our model architecture, <font color=red>we use the Transformer</font>, This model choice provides us with a more structured memory for <font color=red>handling long-term dependencies in text</font>, compared to alternatives like recurrent networks, <font color=red>resulting in robust transfer performance across diverse tasks. </font>


5. We evaluate our approach on four types of language understanding tasks:
    - natural language inference;
    - question answering;
    - semantic similarity;
    - text classification.

### 2.2 Related Work

#### Semi-supervised learning for NLP

Over the last few years, researchers have demonstrated the benefits of using word embeddings, which are trained on unlabeled corpora, to improve performance on a variety of tasks. <font color=blue>These approaches, however, mainly transfer word-level information</font>, whereas <font color=red>we aim to capture higher-level semantics.</font>


#### Unsupervised pre-training

1. Unsupervised pre-training is a special case of semi-supervised learning where <font color=red>the goal is to find a good initialization point instead of modifying the supervised learning objective.</font>


2. Subsequent research demonstrated that <font color=red>pre-training acts as a regularization scheme, enabling better generalization in deep neural networks.</font>


3. The closest line of work to ours involves pre-training a neural network using a language modeling objective and then fine-tuning it on a target task with supervision. 


4. Our choice of **transformer** networks allows us to **capture longer-range linguistic structure**, as demonstrated in our experiments.


5. Other approaches <font color=blue>use **hidden representations** from a pre-trained language or machine translation model</font> as auxiliary features while training a supervised model on the target task. 


#### Auxiliary training objectives

1. Adding auxiliary unsupervised training objectives is an alternative form of semi-supervised learning. 


2. Our experiments also use an auxiliary objective, but as we show, <font color=blue>unsupervised pre-training already learns several linguistic aspects relevant to target tasks.</font>

### 2.3 Framework

#### 2.3.1 Unsupervised pre-training

<table>
    <td><img src='attachment/47. GPT_Framework.png' style='zoom:50%'/></td>
    <td><img src='attachment/47. GPT_Pre-training_and_fine_tuning.png' style='zoom:50%'/></td>
</table>

给定语料库的token集合 $\color{MediumOrchid}{\mathcal{U}=\left\{u_{1}, \ldots, u_{n}\right\}}$，使用标准的语言模型极大化似然函数：

$$
\color{MediumOrchid}{L_{1}(\mathcal{U})=\sum_{i} \log P\left(u_{i} \mid u_{i-k}, \ldots, u_{i-1} ; \Theta\right)} \tag{1}
$$

- $k:$ size of the context window

GPT使用Multi-layer Transformer Decoder来训练语言模型: 

$$
\begin{eqnarray}
\color{red}{h_{0}} &=& \color{red}{UW_{e} + W_{p}~~ ???} \\
h_{l} &=& \text{transformer_block}(h_{l-1}) \quad \forall l \in [1, n]  \tag{2}\\
P(u) &=& \text{softmax}(h_{n}W_{e}^{\mathrm{T}})
\end{eqnarray}
$$

- $U=(u_{-k}, \cdots, u_{-1}):$ context vector of tokens;
- $n:$ number of layers;
- $W_{e}:$ token embedding matrix;
- $W_{p}:$ position embedding matrix.

#### 2.3.2 Supervised fine-tuning

设有标注数据集 $\mathcal{C}$，每一个样本由输入序列 $x^{1}, \cdots, x^{m}$ 和标签 $y$ 组成。

将输入序列传入 pre-trained 模型中，取最后一层transformer block的激活项 $h_{l}^{m}$ ，送进一个线性输出层，预测 $y$:

$$
P\left(y \mid x^{1}, \ldots, x^{m}\right)=\operatorname{softmax}\left(h_{l}^{m} W_{y}\right) \tag{3}
$$

目标是最大化如下目标函数: 

$$
L_{2}(\mathcal{C})=\sum_{(x, y)} \log P\left(y \mid x^{1}, \ldots, x^{m}\right) \tag{4}
$$

最终和$L_{1}$一起优化，目标函数为:

$$
L_{3}(\mathcal{C})=L_{2}(\mathcal{C})+\lambda * L_{1}(\mathcal{C}) \tag{5}
$$


加上fine-tuning的目标函数一起优化，可以:
- improving generalization of the supervised model;
- accelerating convergence. 

# 3. Transformer-XL

- Segment-level Recurrence
- Relative Positional Embedding

在NLP领域，处理语言建模问题有两种最先进的架构：RNN和Transformer。RNN按照序列顺序逐个学习输入的单词或字符之间的关系；而Transformer则接收一整段序列，然后使用self-Attention机制来学习它们之间的依赖关系。

但它们都有一个共同不足之处: unable to model dependencies longer than a fixed length.

Transformer-XL <font color='#c63c26'>同时结合了RNN序列建模和Transformer自注意力机制的优点，在输入数据的每个片段上使用Transformer的Self-Attention模块，并使用循环机制来学习连续段之间的依赖关系。</font>

### 3.1 vanilla Transformer

<img src='attachment/47. Vanilla Transformer.png' style='zoom:50%'/>

Al-Rfou等人基于Transformer提出了vanilla model，它根据之前的字符预测片段中的下一个字符。例如，<font color=MediumOrchid>它使用$x_{1}, x_{2}, \cdots, x_{n-1}$预测字符$x_{n}$，而在$x_{n}$之后的序列则被mask掉。</font>论文中使用64层模型，并仅限于处理512个字符这种相对较短的输入，因此它将输入分成段，并分别从每个段中进行学习，如上图所示。在Evaluation阶段，该模型会在每一步中将输入向右移动一个字符，以此实现对单个字符的预测。


但vanilla model仍有些缺点:

1. 因为segments之间独立训练，所以不同的token之间，最长的依赖关系，就取决于segment的长度；


2. 出于效率考虑，在划分segments的时候，不考虑句子的自然边界，而是根据固定的长度来划分序列，导致分割出来的segments在语义上是不完整的。（context fragmentation problem）；


3. 推理速度慢: 在Evaluation阶段，一般取最后一个位置的隐向量作为输出。为了充分利用上下文关系，在每做完一次预测之后，就对整个序列向右移动一个位置，再做一次计算，如上图（b）所示，则导致计算效率非常低。

### 3.2 Segment-Level Recurrence with State Reuse

<img src='attachment/47. Transformer-XL.png' style='zoom:50%'/>

Transformer-XL在<font color=MediumOrchid>对当前segment进行处理的时候，缓存并利用上一个segment中所有layer的隐向量序列，而且上一个segment的所有隐向量序列只参与前向计算，不再进行反向传播，这就是所谓的segment-level recurrence。</font>

#### 符号说明:

两个连续的segments表示为 $s_{\tau}=\left[x_{\tau, 1}, x_{\tau, 2}, \ldots, x_{\tau, L}\right],\ s_{\tau+1}=\left[x_{\tau+1, 1}, x_{\tau+1, 2}, \ldots, x_{\tau+1, L}\right], \ L$是序列长度；


假设整个模型中，包含$~N~$层Transformer-block，那么每个segment中就有$~N~$组长度为$L$的隐向量序列；


$\mathbf{h}_{\tau}^{n} \in \mathbb{R}^{L \times d}$——表示第$~\tau~$个segment的第$~n~$层隐向量序列；


$\text{SG}$是stop-gradient，不在对$~s_{\tau}$ 的隐向量做反向传播；


$\widetilde{\mathbf{h}}_{\tau+1}^{n-1}$ 是对两个隐向量序列沿长度方向的拼接，\[\]内两个隐向量的维度都是$L \times d$，拼接之后的向量维度是 $2L \times d$;


$\mathbf{q}$ 的计算方式不变，只使用当前segment中隐向量，计算得到的$\mathbf{q}$序列长度仍是$L$；


$\mathbf{k}, \mathbf{v}$采用拼接之后的$\widetilde{\mathbf{h}}$来计算，计算出来的序列长度是$2L$;


Transformer的输出隐向量序列长度取决于query的序列长度，而不是key和value.

$$
\begin{array}{l}
\widetilde{\mathbf{h}}_{\tau+1}^{n-1}=\left[\mathrm{SG}\left(\mathbf{h}_{\tau}^{n-1}\right) \circ \mathbf{h}_{\tau+1}^{n-1}\right] \\
\mathbf{q}_{\tau+1}^{n}, \mathbf{k}_{\tau+1}^{n}, \mathbf{v}_{\tau+1}^{n}=\mathbf{h}_{\tau+1}^{n-1} \mathbf{W}_{q}^{\top}, \widetilde{\mathbf{h}}_{\tau+1}^{n-1} \mathbf{W}_{k}^{\top}, \widetilde{\mathbf{h}}_{\tau+1}^{n-1} \mathbf{W}_{v}^{\top} \\
\mathbf{h}_{\tau+1}^{n}= \text{Transformer-Layer} \left(\mathbf{q}_{\tau+1}^{n}, \mathbf{k}_{\tau+1}^{n}, \mathbf{v}_{\tau+1}^{n}\right)
\end{array}
$$

训练和预测过程如Fig2所示。需注意的一点: <font color=MediumOrchid>在当前segment中，第$~n~$层的每个隐向量的计算，除了依赖当前位置的下一层隐向量，还与前$L-1$个位置的隐向量存在依赖关系，而且每往下走一层，依赖关系长度都会增加$(L-1)$，所以最长的依赖关系是$N(L-1)$。</font>在对长文本进行计算的时候，可以缓存上一个segment的隐向量的结果，不必重复计算，大幅提高计算效率。


上文中，我们只保存了上一个segment，实际操作的时候，可以保存尽可能多的segments，只要内存或者显存放得下。论文中的试验在训练的时候，只缓存一个segment，在预测的时候，会缓存多个segments。

### 3.3 Relative Positional Encodings

Transformer-XL放弃了vanilla transformer绝对位置编码，而采用相对位置编码。<font color=MediumOrchid>具体地，在计算Attention Score的时候，只考虑query向量与key向量的相对位置关系，</font>并且将这种相对位置关系，加入到每一层Transformer的Attention的计算中。


Vanilla Transformer中位置Embedding的计算公式如下: 

$$
\begin{aligned} 
\mathbf { A } _ { i , j } ^ { \mathrm { abs } } 
& = \left\{ \mathbf { W }_{ q } \left( \mathbf { E }_{ x_{ i } } + \mathbf { U }_{ i } \right) \right\}^{\top} 
\left\{ \mathbf { W }_{ k } \left( \mathbf { E }_{ x_{ j } } + \mathbf { U }_{ j } \right) \right\}\\
\quad \\
& = \underbrace { \mathbf { E } _ { x _ { i } } ^ { \top } \mathbf { W } _ { q } ^ { \top } \mathbf { W } _ { k } \mathbf { E } _ { x _ { j } } } _ { ( a ) } + \underbrace { \mathbf { E } _ { x _ { i } } ^ { \top } \mathbf { W } _ { q } ^ { \top } \mathbf { W } _ { k } \mathbf { U } _ { j } } _ { ( b ) }  + \underbrace { \mathbf { U } _ { i } ^ { \top } \mathbf { W } _ { q } ^ { \top } \mathbf { W } _ { k } \mathbf { E } _ { x _ { j } } } _ { ( c ) } + \underbrace { \mathbf { U } _ { i } ^ { \top } \mathbf { W } _ { q } ^ { \top } \mathbf { W } _ { k } \mathbf { U } _ { j } } _ { ( d ) } 
\end{aligned}
$$

而Transformer-XL中使用相对位置计算attention score的公式如下: 

$$
\begin{aligned} 
\mathbf { A } _ { i , j } ^ { \mathrm { rel } } 
& = \underbrace { \mathbf { E } _ { x _ { i } } ^ { \top } \mathbf { W } _ { q } ^ { \top } \mathbf { W } _ { k,E } \mathbf { E } _ { x _ { j } } } _ { ( a ) } + \underbrace { \mathbf { E } _ { x _ { i } } ^ { \top } \mathbf { W } _ { q } ^ { \top } \mathbf { W } _ { k,R } \color{DeepSkyBlue}{ \mathbf { R } _ { i-j }} } _ { ( b ) }  + \underbrace { \color{red}{u ^ { \top }} \mathbf { W } _ { k,E }  \mathbf { E } _ { x _ { j } } } _ { ( c ) } + \underbrace { \color{red}{ v ^ { \top }} \mathbf { W } _ { k,R }  \color{DeepSkyBlue}{ \mathbf { R } _ { i-j }} } _ { ( d ) } 
\end{aligned}
$$

其中: 
- $\color{red}{u,v}$ 是trainable parameters;  


- $\mathbf{W}_{k,E}$ 用于生成基于内容的key向量；


- $\mathbf{W}_{k,R}$ 用于生成基于位置的key向量；


- $\mathbf{R} \in \mathbb{R}^{L_{max} \ \  \times d}$，第$~i~$行表示相对位置间隔为$~i~$的位置向量。论文中强调$\mathbf{R}$采用正弦函数生成，而不是通过学习得到的。

最后，对于一个$N$ 层的 single attention head 的Transformer-XL的计算公式如下: 

$\text{For} \quad  n=1,\cdots, N:$ 

<img src='attachment/47. Transformer-XL_computational_procedure.png' style='zoom:50%' align='left'/>

# 3. XLNet

*XLNet: Generalized Autoregressive Pretraining for Language Understanding*

Part of Advances in Neural Information Processing Systems 32 (<font color=red>NeurIPS 2019</font>)

**Authors:**

Zhilin Yang, Zihang Dai, Yiming Yang, Jaime Carbonell, Russ R. Salakhutdinov, Quoc V. Le

## 3.1 AR vs. AE

<img src='attachment/47. AR&AE.png' style='zoom:50%'/>

### 3.1.1 Auto-Regression LM

ELMo(2018.03)/BERT(2018.10.11)出来之前，大家通常讲的Language Model其实是根据上文内容预测下一个可能跟随的单词，就是常说的自左向右的Language Model，或者反过来也行。这种Language Model被称为自回归语言模型（Auto-Regression LM）。GPT是典型的AR LM。ELMo尽管看上去是利用了上文，也利用了下文，但是本质上是Auto-Regression LM。<font color=MediumOrchid>ELMo是分别做了两个方向的自回归LM，然后把LSTM的两个方向的隐状态拼接到一起，来体现双向语言模型这个事情。所以本质上认识Auto-Regression LM</font>

给定文本序列 $\pmb{x}=\left[x_{1}, \ldots, x_{T}\right]$，Language Model的目标是调整参数使得训练数据上的似然函数最大: 

$$
\max _{\theta} \log p_{\theta}(\pmb{x}) \color{red}{=} \sum_{t=1}^{T} \log p_{\theta}\left(x_{t} \mid \pmb{x}_{<t}\right)=\sum_{t=1}^{T} \log \frac{\exp \left(h_{\theta}\left(\pmb{x}_{1: t-1}\right)^{T} e\left(x_{t}\right)\right)}{\sum_{x^{\prime}} \exp \left(h_{\theta}\left(\pmb{x}_{1: t-1}\right)^{T} e\left(x^{\prime}\right)\right)}
$$

- $\pmb{x}_{<t}$: 表示$~t~$时刻之前的所有$x$，也就是 $\pmb{x}_{1:t-1}$；
- $h_{\theta}\left(\pmb{x}_{1: t-1}\right)$ 是RNN或者Transformer编码的$t$时刻之前的隐状态；
- $e(x)$: 是词$x$的Embedding;
- '=' 等号成立，是因为使用了chain rule.


Auto-Regression LM的优点:
- No discrepancy;
- 考虑依赖关系

Auto-Regression LM的缺点: 
- <font color='#c63c26'>是无法同时利用上下文的信息。</font>

### 3.1.2 Auto-Encoder LM

BERT是一种典型的自编码语言模型（Auto-Encoder LM）。它通过将序列$~\pmb{x}~$中随机挑选15%的Token变成\[MASK\]得到带噪声版本的 $\hat{\pmb{x}}$。假设被Mask的原始值为$\bar{\pmb{x}}$，那么BERT希望尽量根据上下文恢复（猜测）出原始值，也就是: 

$$
\max _{\theta} \log p_{\theta}(\overline{\mathbf{x}} \mid \hat{\mathbf{x}}) \color{red}{ \approx } \sum_{t=1}^{T} m_{t} \log p_{\theta}\left(x_{t} \mid \hat{\mathbf{x}}\right)=\sum_{t=1}^{T} m_{t} \log \frac{\exp \left(H_{\theta}(\mathbf{x})_{t}^{T} e\left(x_{t}\right)\right)}{\sum_{x^{\prime}} \exp \left(H_{\theta}(\mathbf{x})_{t}^{T} e\left(x^{\prime}\right)\right)}
$$

- $m_{t}=1$ 表示$t$时刻是一个MASK，需要恢复；
- $H_{\theta}$ 是一个Transformer，它把长度为$T$的序列$\pmb{x}$映射为隐状态的序列 $H_{\theta}(\mathbf{x})=\left[H_{\theta}(\mathbf{x})_{1}, H_{\theta}(\mathbf{x})_{2}, \ldots, H_{\theta}(\mathbf{x})_{T}\right]$
- '$\color{red}{\approx}$' ，是因为引入了条件独立的假设(Independent Assumption)，$P(New ~ York|is,a,city) \color{red}{\approx} P(New|is,a,city)\cdot P(York|is,a,city)$


Auto-Encoder LM能比较自然地融入双向语言模型，同时看到被预测单词的上文和下文，这是优点。但是，<font color='#c63c26'>在输入侧引入\[MASK\]标记，导致Pre-training和Fine-tuning阶段不一致的问题，因为Fine-tuning阶段是看不到\[MASK\]标记的。</font>

<font color='#c63c26'>XLNet 的出发点就是：能否融合自回归 LM 和 DAE LM 两者的优点。具体来说就是，站在 AR 的角度，如何引入和双向语言模型等价的效果.</font>

## 3.2 Permutation Language Model

<img src='attachment/47. Illustration of permutation LM.png' style='zoom:50%'/>

#### 要点：
- Sample a factorization order; 

- Determine the attention masks based on the order;

- Optimize a standard language modeling objective:

$$
\max _{\theta} \mathbb{E}_{\mathbf{z} \sim \mathcal{Z}_{T}}\left[\sum_{t=1}^{T} \log p_{\theta}\left(x_{z_{t}} \mid \mathbf{x}_{\mathbf{z}_{<t}}\right)\right]
$$

- Benifits: 
     - Autoregressive, avoding disadvantages of AE;
     - Able to model bidirectional context
     


具体实现方式:  <font color=blue>通过随机取一句话排列的一种，然后将末尾一定量的词给“遮掩”（和BERT的[MASK]有些不同）掉，最后用Auto-Regression的方式来按照这种排列方式依次预测被“遮掩”掉的词。</font>


<img src='attachment/47. Permutation LM demo.png' style='zoom:50%'/>

<font color=red>最后“遮掩”的token长度怎么选择呢？</font>
作者设了一个超参数$K$，$K$等于总长度除以需要预测的个数。以上图为例，中长为7，需要预测的长度为2，于是$K=7/2$。论文中实验得出的最佳$K$值介于6和7之间。如果去$K$的导数（即$\frac{1}{6}, \frac{1}{7}$），转化为百分比为（14.3%，16.7%）之间。而BERT中将Token替换为\[MASK\]的比列就是15%，二者之间应该有本质上的联系。


<font color=red>关于句子排列的采样：</font>对于一个长度为$T$的句子，有$T!$中排列，如果遍历每种排列，是不现实的。用 $\mathcal{Z}_{T}$ 表示所有排列组成的集合，$\mathcal{z}$ 表示从$\mathcal{Z}_{T}$采样得到一种排序，记为$\mathcal{z} \sim \mathcal{Z}_{T}$

XLNet并不是打乱输入句子的顺序，而是通过Transformer的Attention Masks来巧妙实现的。

<img src='attachment/47. XLNet Attention masks.png' style='zoom:50%'/>

## 3.3 Reparameterization

Permutation Language Model的思想很简单，但如果还是用standard Transformer parameterization，就会有问题，standard Transformer parameterization公式为: 

$$
\max _{\theta} \ \log p_{\theta}(\pmb{x}) \color{red}{=} \sum_{t=1}^{T} \log p_{\theta}\left(x_{t} \mid \pmb{x}_{<t}\right)=\sum_{t=1}^{T} \log \frac{\exp \left(h_{\theta}\left(\pmb{x}_{1: t-1}\right)^{T} e\left(x_{t}\right)\right)}{\sum_{x^{\prime}} \exp \left(h_{\theta}\left(\pmb{x}_{1: t-1}\right)^{T} e\left(x^{\prime}\right)\right)}
$$

举例来说: 

假设输入的句子是 "New York is a city"，它由两种排列： 
1. is a city New York ——> \[3,4,5,1,2\]
2. is a city York New ——> \[3,4,5,2,1\]

对于第1种排列，假设我们要预测$z_{4} = New$，则有: 

$$
p_{\theta}\left( \text{New} \mid \text{is, a, city} \right) =  \frac{\exp 
\left \{ h_{\theta} \left( \text{is, a, city} \right)^{\mathrm{T}} \cdot e(\text{New}) \right\} }{ \sum_{x^{\prime}} \exp 
\left \{ h_{\theta} \left( \text{is, a, city} \right)^{\mathrm{T}} \cdot e(x^{\prime}) \right\} }
$$

同理，对于第2中排列，假设我们要预测$z_{4} = New$，同样有: 

$$
p_{\theta}\left( \text{New} \mid \text{is, a, city} \right) =  \frac{\exp 
\left \{ h_{\theta} \left( \text{is, a, city} \right)^{\mathrm{T}} \cdot e(\text{New}) \right\} }{ \sum_{x^{\prime}} \exp 
\left \{ h_{\theta} \left( \text{is, a, city} \right)^{\mathrm{T}} \cdot e(x^{\prime}) \right\} }
$$

上面两个公式得到的概率是相等的，但是对于两种排列，它们的概率应该是不相等的，而问题的原因在于<font color=red>$h_{\theta}\left(\pmb{x}_{1: t-1}\right)$没有建模位置信息。</font>

为了解决上述问题，XLNet提出了一种新的参数化表示方法: 

$$
p_{\theta}\left(X_{z_{t}}=x \mid \mathbf{x}_{z_{<t}}\right)=\frac{\exp \left(e(x)^{\top} g_{\theta}\left(\mathbf{x}_{\mathbf{z}_{<t}} \ , z_{t}\right)\right)}{\sum_{x^{\prime}} \exp \left(e\left(x^{\prime}\right)^{\top} g_{\theta}\left(\mathbf{x}_{\mathbf{z}_{<t}} \ ,  z_{t}\right)\right)}
$$

- $\mathbf{x}_{\mathbf{z}_{<t}}$  表示$\mathbf{x}_{1:t-1}$的意思（包含了内容和位置信息）
- $z_{t}$ 表示$x_{t}$（预测）的位置信息

## 3.4 Two-Stream Self-Attention

对于3.3中提出的参数化表示，该用什么模型来表示$g_{\theta} \left(\mathbf{x}_{\mathbf{z} {<t}},z_{t} \right)$呢？

$g_{\theta}$需满足下面两个条件: 

1. 预测$~x_{z_{t}}~$ 时，只能使用位置信息$z_{t}$，而不能使用内容信息$x_{z_{t}}$；
2. 而预测其它tokens $x_{z_{j}}(j > t)$时，需要包含内容信息$x_{z_{t}}$

对于上面的两点要求，普通的Transformer Self-Attention是不能满足的，举例说明: 

<img src='attachment/47. XLNet Original Transformer Attention.png' style='zoom:40%'/>

为解决上述问题，XLNet引入了<font color=red>Two-Stream Self-Attention</font>的设计: 

<img src='attachment/47. Two-Stream Self-Attention.png' style='zoom:50%'/>


从上图可以看到，在计算Attention时，引入了两个Stream，也就是两个隐状态: 

- 内容隐状态 $h_{\theta}(\mathbf{x}_{z<t})$，简写为$h_{z_{t}}$，它和标准的Transformer一样，既编码上下文内容，也编码$\mathbf{x}_{z_{t}}$的内容；


- 查询隐状态 $g_{\theta}(\mathbf{x}_{z<t},z_{t})$，简写为$g_{z_{t}}$，它只编码上下文和要预测的位置$z_{t}$，但不包含$\mathbf{x}_{z_{t}}$


#### 计算过程——初始化

1. 把查询隐状态$g_{i}^{(0)}$初始化为一个变量 $w$；


2. 把内容隐状态$h_{i}^{(0)}$初始化为token的Embedding $e(x_{i})$


这里上标(0)表示第0层（并不存在，只是用于计算第1层）


#### 计算过程——逐层计算

$$
\begin{align}
\text{Query Stream}&: \quad g_{z_{t}}^{(m)} \leftarrow \text{Attention} \left(Q=\color{red}{g_{z_{t}}^{(m-1)}}, K V=h_{\color{red}{z_{<t}}}^{(m-1)} ; \theta\right) \\
\text{Content Stream}&: \quad h_{z_{t}}^{(m)} \leftarrow \text{Attention} \left(Q=\color{red}{h_{z_{t}}^{(m-1)}}, K V=h_{\color{red}{z_{\leq t}}}^{(m-1)} ; \theta\right)
\end{align}
$$

- Query Stream: use $z_{t}$ but cannot see $\mathbf{x}_{z_{t}}$


- Content Stream: use both $z_{t}$ and $\mathbf{x}_{z_{t}}$


上面的梯度更新和Standard Transformer Self-Attention一样，在Fine-tuning时，可以丢弃掉Query Stream而只用 Content Stream。最后在计算公式的时候，我们可以用上面一层的Query向量$g_{z_{t}}^{(M)}$。

## 3.5 XLNet vs. BERT

假设输入Sentence='New York is a city'. 并且\[New, York\]是要预测的目标，则目标函数是 $\text{maximize} ~ \log p(\text{New York }\mid \text{ is, a, city)}$;

假设XLNet采样得到的句子排列是\[is, a, city, New, York\]，则BERT和XLNet的目标函数分别是:  

$$
\begin{align}
\mathcal{J}_{BERT} &= \log p\left(\text{New} \mid \text{is a city} \right) + \log p\left(\text{York} \mid \text{is a city} \right) \\
\mathcal{J}_{XLNet} &= \log p\left(\text{New} \mid \text{is a city} \right) + \log p\left(\text{York} \mid \color{red}{\text{New}}, \text{is a city} \right)
\end{align}
$$

可以看到，XLNet可以捕捉(New, York)词对之间的相互依赖关系，而BERT不能。

## 3.6 Summary

<img src='attachment/47. XLNet Summary.png' style='zoom:50%'/>